# Titanic Project Example Walk Through 
In this notebook, I hope to show how a data scientist would go about working through a problem. The goal is to correctly predict if someone survived the Titanic shipwreck. I thought it would be fun to see how well I could do in this compention without deep learning. 

*The accompanying video is located here:* https://www.youtube.com/watch?v=I3FBJdiExcg

**Best results : 79.425 % accuracy (Top 12%)**

## Overview 
### 1) Understand the shape of the data (Histograms, box plots, etc.)

### 2) Data Cleaning 

### 3) Data Exploration

### 4) Feature Engineering 

### 5) Data Preprocessing for Model

### 6) Basic Model Building 

### 7) Model Tuning 

### 8) Ensemble Modle Building 

### 9) Results 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):# os.walk() returns root, dirs and files
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# dirname(root) is the directory of the current folder itself.当前文件夹‘/kaggle/input’的路径
# _(dirs) are the sub folders inside the current folder. 该文件夹'kaggle/input'的子目录名list
# filenames (files) are the file names inside the current. 该文件夹‘kaggle/input’的文件名list


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Here we import the data. For this analysis, we will be exclusively working with the Training set. We will be validating based on data from the training set as well. For our final submissions, we will make predictions based on the test set. 

In [ ]:
training = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

training['train_test'] = 1 # to isolate data from training to testing
test['train_test'] = 0
test['Survived'] = np.NaN
all_data = pd.concat([training,test])

%matplotlib inline
all_data.columns

## Project Planning
When starting any project, I like to outline the steps that I plan to take. Below is the rough outline that I created for this project using commented cells. 

In [ ]:
# Understand nature of the data .info() .describe()
# Histograms and boxplots 
# Value counts 
# Missing data 
# Correlation between the metrics 
# Explore interesting themes 
    # Wealthy survive? 
    # By location 
    # Age scatterplot with ticket price 
    # Young and wealthy Variable? 
    # Total spent? 
# Feature engineering 
# preprocess data together or use a transformer? 
    # use label for train and test   
# Scaling?

# Model Baseline 
# Model comparison with CV 

## Light Data Exploration
### 1) For numeric data 
* Made histograms to understand distributions (other options: boxplot, violinplot)
* Corrplot 
* Pivot table comparing survival rate across numeric variables 


### 2) For Categorical Data 
* Made bar charts to understand balance of classes (other options, countplot/histogram )
* Made pivot tables to understand relationship with survival 

In [ ]:
#quick look at our data types & null counts 
training.info()

In [ ]:
# to better understand the numeric data, we want to use the .describe() method. This gives us an understanding of the central tendencies of the data 
training.describe()

In [ ]:
#quick way to separate numeric columns
training.describe().columns

In [ ]:
# look at numeric and categorical values separately 
df_num = training[['Age','SibSp','Parch','Fare']] # histogram/box plot to find distribution
df_cat = training[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']] #count plot/value count to find frequency

In [ ]:
#distributions for all numeric variables 
for i in df_num.columns:
    plt.hist(df_num[i])
    plt.title(i)
    plt.show()

Perhaps we should take the non-normal distributions and consider normalizing them? Apparently, except for age, the rest three numerical variables do not display a normal distribution. 

In [ ]:
print(df_num.corr())
sns.heatmap(df_num.corr())

When running linear regression, we want to avoid adopting variables that have multicolinearity which means there exist strong correlations between one another. 

Here, both slipsp and partch seem to have a slight negative correation with age, while parch and sibsp appear to have a moderately postive correlation with each other.

In [ ]:
# compare survival rate across Age, SibSp, Parch, and Fare 
pd.pivot_table(training, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])

# 等价pivot table和group by
# pd.pivot_table(df,index=[字段1],values=[字段2],aggfunc=[函数],fill_value=0)默认mean函数
# df.groupby([字段1])[字段2].agg(函数).fillna(0) 


My personal attempt to achieve the same effect generated by pivot_table() with "groupby().agg()" function.


In [ ]:

training.groupby(["Survived"])[['Age','SibSp','Parch','Fare']].mean()

In [ ]:
# use bar plot/countplot to measure the count differences among categorical variables
for i in df_cat.columns:
    sns.barplot(df_cat[i].value_counts().index,df_cat[i].value_counts()).set_title(i)
    plt.show()\
    

Cabin and ticket graphs are very messy. This is an area where we may want to do some feature engineering! 

My personal attempt to achieve the same effect generated by sns.barplot(x.value_counts().index, x.value_counts()) with "sns.countplot()" function.

In [ ]:
# try to integrate all four graphs in one figure
fig, axss = plt.subplots(2,3,figsize=[20,10])
sns.countplot(x=df_cat.columns[0], data=df_cat, ax=axss[0][0])
sns.countplot(x=df_cat.columns[1], data=df_cat, ax=axss[0][1])
sns.countplot(x=df_cat.columns[2], data=df_cat, ax=axss[0][2])
sns.countplot(x=df_cat.columns[3], data=df_cat, ax=axss[1][0])
sns.countplot(x=df_cat.columns[4], data=df_cat, ax=axss[1][1])
sns.countplot(x=df_cat.columns[5], data=df_cat, ax=axss[1][2])


In [ ]:
# Comparing survival and each of these categorical variables 
print(pd.pivot_table(training, index = 'Survived', columns = 'Pclass', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Sex', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Embarked', values = 'Ticket' ,aggfunc ='count'))

## Feature Engineering 
### 1) Cabin - Simplify cabins (evaluated if cabin letter (cabin_adv) or the purchase of tickets across multiple cabins (cabin_multiple) impacted survival)

### 2) Tickets - Do different ticket types impact survival rates?

### 3) Does a person's title relate to survival rates? 

In [ ]:
df_cat.Cabin

# part 1
# find a potential feature "cabin_multiple" and create a new column "cabin_multiple"
training['cabin_multiple'] = training.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))

# after looking at this, we may want to look at cabin by letter or by number. Let's create some categories for this 
# letters 
# multiple letters 
training['cabin_multiple'].value_counts()

In [ ]:
# compare survival rate by cabin_multiple
pd.pivot_table(training, index = 'Survived', columns = 'cabin_multiple', values = 'Ticket' ,aggfunc ='count')

In [ ]:
#creates categories based on the cabin letter (n stands for null)
#in this case we will treat null values like it's own category

# part 2
# find a potential feature "cabin_adv" and create a new column "cabin_adv"
training['cabin_adv'] = training.Cabin.apply(lambda x: str(x)[0])

# n represents "null" and it seems like there are lots of null for cabin values


In [ ]:
#comparing surivial rate by cabin
print(training.cabin_adv.value_counts())
pd.pivot_table(training,index='Survived',columns='cabin_adv', values = 'Name', aggfunc='count')

Clearly, people who don't have specific cabin names have much higher death rate. Those with specific 

In [ ]:
# find a potential feature "ticket types" and create a new pair of column "numeric_ticket" and "ticket_letters"

#understand ticket values better 
#numeric vs non numeric 
training['numeric_ticket'] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
training['ticket_letters'] = training.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)

# here the last component of ticket, as seperated by " ", is always digit. Thus we are excluding it. 

In [ ]:
training['numeric_ticket'].value_counts()

In [ ]:
training['ticket_letters'].head(20)

In [ ]:
#lets us view all rows in dataframe through scrolling. This is for convenience 
pd.set_option("max_rows", None)
training['ticket_letters'].value_counts()


In [ ]:
#difference in numeric vs non-numeric tickets in survival rate 
pd.pivot_table(training,index='Survived',columns='numeric_ticket', values = 'Ticket', aggfunc='count')

In [ ]:
#survival rate across different tyicket types 
pd.pivot_table(training,index='Survived',columns='ticket_letters', values = 'Ticket', aggfunc='count')

In [ ]:
#feature engineering on person's title 
training.Name.head(50)
training['name_title'] = training.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
#mr., ms., master. etc

In [ ]:
training['name_title'].value_counts()

## Data Preprocessing for Model 
### 1) Drop null values from Embarked (only 2) 

### 2) Include only relevant variables (Since we have limited data, I wanted to exclude things like name and passanger ID so that we could have a reasonable number of features for our models to deal with) 
Variables:  'Pclass', 'Sex','Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'cabin_adv', 'cabin_multiple', 'numeric_ticket', 'name_title'

### 3) Do categorical transforms on all data. Usually we would use a transformer, but with this approach we can ensure that our traning and test data have the same colums. We also may be able to infer something about the shape of the test data through this method. I will stress, this is generally not recommend outside of a competition (use onehot encoder，or ordinal encoder). 

### 4) Impute data with mean for fare and age (Should also experiment with median) 

### 5) Normalized fare using logarithm to give more semblance of a normal distribution (log normalization > transform into 0 - 1 range)

### 6) Scaled data 0-1 with standard scaler (standardization > turn to normal distribution)


In [ ]:
#create all categorical variables that we did above for both training and test sets 
all_data['cabin_multiple'] = all_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
all_data['cabin_adv'] = all_data.Cabin.apply(lambda x: str(x)[0])
all_data['numeric_ticket'] = all_data.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
all_data['ticket_letters'] = all_data.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)
all_data['name_title'] = all_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

#impute nulls for continuous data 
#all_data.Age = all_data.Age.fillna(training.Age.mean())
all_data.Age = all_data.Age.fillna(training.Age.median())
#all_data.Fare = all_data.Fare.fillna(training.Fare.mean())
all_data.Fare = all_data.Fare.fillna(training.Fare.median())

#drop null 'embarked' rows. Only 2 instances of this in training and 0 in test 
all_data.dropna(subset=['Embarked'],inplace = True)

#tried log normalization of sibsp (not used)
all_data['norm_sibsp'] = np.log(all_data.SibSp+1)
all_data['norm_sibsp'].hist()
 
# log norm of fare (used)
all_data['norm_fare'] = np.log(all_data.Fare+1)
all_data['norm_fare'].hist()

# converted fare to category for pd.get_dummies()
all_data.Pclass = all_data.Pclass.astype(str)

#created dummy variables from categories (also can use OneHotEncoder from sklearn.preprocessing)
all_dummies = pd.get_dummies(all_data[['Pclass','Sex','Age','SibSp','Parch','norm_fare','Embarked','cabin_adv','cabin_multiple','numeric_ticket','name_title','train_test']])

all_dummies.head(20)


In [ ]:
#Split to train test again
X_train = all_dummies[all_dummies.train_test == 1].drop(['train_test'], axis =1)
X_test = all_dummies[all_dummies.train_test == 0].drop(['train_test'], axis =1)


y_train = all_data[all_data.train_test==1].Survived # get Survived column from training data only
y_train.shape

In [ ]:
# Scale data to normal distribution (zero mean, unit varience) to avoid extreme values with huge varience dominating parameters estimation of function 
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
all_dummies_scaled = all_dummies.copy()
all_dummies_scaled[['Age','SibSp','Parch','norm_fare']]= scale.fit_transform(all_dummies_scaled[['Age','SibSp','Parch','norm_fare']])
all_dummies_scaled[['Age','SibSp','Parch','norm_fare']].head(20)



In [ ]:
X_train_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 1].drop(['train_test'], axis =1)
X_test_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 0].drop(['train_test'], axis =1)

y_train = all_data[all_data.train_test==1].Survived

## Model Building (Baseline Validation Performance)
Before going further, I like to see how various different models perform with default parameters. I tried the following models using 5 fold cross validation to get a baseline. With a validation set basline, we can see how much tuning improves each of the models. Just because a model has a high basline on this validation set doesn't mean that it will actually do better on the eventual test set. 

- Naive Bayes (72.6%) 
- Logistic Regression (82.1%) 
- Decision Tree (77.6%) 
- K Nearest Neighbor (80.5%) 
- Random Forest (80.6%) 
- **Support Vector Classifier (83.2%)**
- Xtreme Gradient Boosting (81.8%)
- Soft Voting Classifier - All Models (82.8%)

The above mentioned classification models all have mean accuracy as their default score. 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
#I usually use Naive Bayes as a baseline for my classification tasks 
gnb = GaussianNB() 
cv = cross_val_score(gnb,X_train_scaled,y_train,cv=5) # by default the score is estimator's default score
print(cv)
print(cv.mean())

Cross validation accelerates the splitting of training and testing data for multiple times. Value of cv decides the times of training as well.

Otherwise, for each training, it requires train_test_split(), model.fit(training), model.score(testing) to generate the accuracy of one-time training. 

In [ ]:
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

Max_iter regulates the maximum iteration frequency for gradient descent to get the minimum loss function.

In [ ]:
dt = tree.DecisionTreeClassifier(random_state = 1)
cv = cross_val_score(dt,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

Setting random_state to a fixed interger ensures a deterministic splitting behaviours during fitting. 

In [ ]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf,X_train,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
svc = SVC(probability = True)
cv = cross_val_score(svc,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

SVM enables classification of data which are linearly seperable, and non-linearly seperate with the kernel trick setting. SVM algorithm finds the hyperplane that maximizes the margin from both tags. 

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state =1)
cv = cross_val_score(xgb,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
#Voting classifier takes all of the inputs and averages the results. For a "hard" voting classifier each classifier gets 1 vote "yes" or "no" and the result is just a popular vote. For this, you generally want odd numbers
#A "soft" classifier averages the confidence of each of the models. If a the average confidence is > 50% that it is a 1 it will be counted as such
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators = [('lr',lr),('knn',knn),('rf',rf),('gnb',gnb),('svc',svc),('xgb',xgb)], voting = 'soft') 

In [ ]:
cv = cross_val_score(voting_clf,X_train_scaled,y_train,cv=5)
print(cv)
print(cv.mean())

In [ ]:
voting_clf.fit(X_train_scaled,y_train)
y_hat_base_vc = voting_clf.predict(X_test_scaled).astype(int)
basic_submission = {'PassengerId': test.PassengerId, 'Survived': y_hat_base_vc}
base_submission = pd.DataFrame(data=basic_submission)
base_submission.to_csv('base_submission.csv', index=False)

## Model Tuned Performance 
After getting the baselines, let's see if we can improve on the indivdual model results!I mainly used grid search to tune the models. I also used Randomized Search for the Random Forest and XG boosted model to simplify testing time. 

|Model|Baseline|Tuned Performance|
|-----|--------|-----------------|
|Naive Bayes| 72.6%| NA|
|Logistic Regression| 82.1%| 82.6%|
|Decision Tree| 77.6%| NA|
|K Nearest Neighbor| 80.5%|83.0%|
|Random Forest| 80.6%| 83.6|
|Support Vector Classifier| 83.2%| 83.2%|
|Xtreme Gradient Boosting| 81.8%| 85.3%|


In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV 

In [ ]:
#simple performance reporting function
def clf_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

In [ ]:
# find names and current values for all parameters for a given estimator
lr = LogisticRegression()
lr.get_params()

In [ ]:
lr = LogisticRegression()
param_grid = {'max_iter' : [2000],
              'penalty' : ['l1', 'l2'],
              'C' : np.logspace(-4, 4, 20),
              'solver' : ['liblinear']}
# C is the inverse of regularization strength.
# np.logspace(start, end, num) generates numbers spaced evently on a log scale (base=10 by default)
# solver defines the algorithm to use in optimization of loss function. 
# liblinear is the only one suitable for both L1 and L2 regularizaton. The others (lbfgs, sag, newton-cg) requires either first derivative and second derivative. 


clf_lr = GridSearchCV(lr, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_lr = clf_lr.fit(X_train_scaled,y_train)
clf_performance(best_clf_lr,'Logistic Regression')
# n_jobs=-1 sets all CPUs to be used concurrently. 
# verbose=True display more massages about cross validation and fitting.

In [ ]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors' : [3,5,7,9],
              'weights' : ['uniform', 'distance'],
              'algorithm' : ['auto', 'ball_tree','kd_tree'],
              'p' : [1,2]}
# weights (uniform: points equally weighed; distance:points weighed by the inverse of their distance > the closer, the greater influencce.)
# algorithm defines the algorithms used to fastly compute the nearest neighbors (ball tree, kd tree are both methods for fast generalizatio of N-point. auto will decide the most appropriate algorithm based on the values passed to the fit.)
# p=1 using manhattan distance, p=2 using euclidean distance

clf_knn = GridSearchCV(knn, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_knn = clf_knn.fit(X_train_scaled,y_train)
clf_performance(best_clf_knn,'KNN')

In [ ]:
svc = SVC(probability = True)
param_grid = tuned_parameters = [{'kernel': ['rbf'], 'gamma': [.1,.5,1,2,5,10],
                                  'C': [.1, 1, 10, 100, 1000]},
                                 {'kernel': ['linear'], 'C': [.1, 1, 10, 100, 1000]},
                                 {'kernel': ['poly'], 'degree' : [2,3,4,5], 'C': [.1, 1, 10, 100, 1000]}]

# C (aka. penalty parameter) defines the inverse of regularization strength. 
# kernal: (linear for linear hyperplane; poly and rbf are non linear hyperplanes)
# gamma is a parameter for non linear hyperplanes, the higher the more exactly fitted for model (overfitting) in the training data set.
# 

clf_svc = GridSearchCV(svc, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_svc = clf_svc.fit(X_train_scaled,y_train)
clf_performance(best_clf_svc,'SVC')

In [ ]:
#Because the total feature space is so large, I used a randomized search to narrow down the paramters for the model. I took the best model from this and did a more granular search 
"""
rf = RandomForestClassifier(random_state = 1)
param_grid =  {'n_estimators': [100,500,1000], 
                                  'bootstrap': [True,False],
                                  'max_depth': [3,5,10,20,50,75,100,None],
                                  'max_features': ['auto','sqrt'],
                                  'min_samples_leaf': [1,2,4,10],
                                  'min_samples_split': [2,5,10]}
                                  
clf_rf_rnd = RandomizedSearchCV(rf, param_distributions = param_grid, n_iter = 100, cv = 5, verbose = True, n_jobs = -1)
best_clf_rf_rnd = clf_rf_rnd.fit(X_train_scaled,y_train)
clf_performance(best_clf_rf_rnd,'Random Forest')

GridSearch experiements all permutation of parameters. RandomizedSearch conducts randome sampling of parameter settings to find the best ones.
"""

In [ ]:
rf = RandomForestClassifier(random_state = 1)
param_grid =  {'n_estimators': [400,450,500,550],
               'criterion':['gini','entropy'],
                                  'bootstrap': [True],
                                  'max_depth': [15, 20, 25],
                                  'max_features': ['auto','sqrt', 10],
                                  'min_samples_leaf': [2,3],
                                  'min_samples_split': [2,3]}
                                  
clf_rf = GridSearchCV(rf, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_rf = clf_rf.fit(X_train_scaled,y_train)
clf_performance(best_clf_rf,'Random Forest')

In [ ]:
best_rf = best_clf_rf.best_estimator_.fit(X_train_scaled,y_train)
feat_importances = pd.Series(best_rf.feature_importances_, index=X_train_scaled.columns)
feat_importances.nlargest(20).plot(kind='barh')

Feature Importance for Tree Models (Random Forest and Gradient Boosting)

Random Forest
1. Gini Importance/Mean Decrease in Impurity 

Measure Gini importance of each node and select nodes (features) with higher Gini importance.

* Gini index: measure data purity of each node, formula = 1-Epi^2 (pi = probability of each feature), (the lower the index value, the higher the purity) 
* Gini decrease: Gini index of each node subtract all of its subnodes' Gini index (weighted)
* Gini importance: sum of Gini decrease (weighted) of all the same nodes across the forest (trees)

Features are selected based on Gini Importance (the measurement of purity of features)

Disadvantage: 
1) It is positively biased against features with more option values
2) For the features with multicolinearity, each of these could be an important feature. However, once one is selected, the others' importance will decrease immediately, making it more difficult to be selected. It is likely to cause misunderstanding that the one selected is very important while the others are not!!


2. Permutation Importance/Mean Decrease in Accuracy (theoretically applicable to all tree-bsaed models)

Randomly permutate the feature order to measure the impact such alternation can bring to the original model performance (i.e. comparison of accuracy before and after)

1) step 1: train a rf model and get accuracy0
2) step 2: randomly permutate feature A and get accuracy 1
3) step 3: (accuracy 0 - accuracy 1)/accuracy 0 to get feature A's importance. 

The higher the importance Feature X has, the bigger the impact there is after the random permutation. 


3. Boruta (theoretically applicable to all tree-based models)

Measure feature importance based on iterative comparison of original features and shadow features, stop when all original features are compared or a predefined rule has been reached. 

1) Create shadow features: add randome interference to the origianl features, then random sampling from extended features to get n number of shadow features.
2) Compare each original feature with the one having the highest importance in shadow feature set to decide whether this original feature is important or not.

Advantage: 
1) Take all features into consideration, regardless of how strong the relevance it has with determining features
2) Enable feature selection through iterations to minimize the error caused by random forest. 

Gradient Boosting:
1. Split-based and Gain-based Measures

Split and gain are both feature attribution methods used to measure the feature importance. Sometimes the results from different methods are inconsistent, SHAP will be used to maintian this consistence.

Split: The number of times when a feature appears as node in the whole tree model. 



In [ ]:
"""xgb = XGBClassifier(random_state = 1)

param_grid = {
    'n_estimators': [20, 50, 100, 250, 500,1000],
    'colsample_bytree': [0.2, 0.5, 0.7, 0.8, 1],
    'max_depth': [2, 5, 10, 15, 20, 25, None],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5, 2],
    'subsample': [0.5,0.6,0.7, 0.8, 0.9],
    'learning_rate':[.01,0.1,0.2,0.3,0.5, 0.7, 0.9],
    'gamma':[0,.01,.1,1,10,100],
    'min_child_weight':[0,.01,0.1,1,10,100],
    'sampling_method': ['uniform', 'gradient_based']
}

#clf_xgb = GridSearchCV(xgb, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
#best_clf_xgb = clf_xgb.fit(X_train_scaled,y_train)
#clf_performance(best_clf_xgb,'XGB')
clf_xgb_rnd = RandomizedSearchCV(xgb, param_distributions = param_grid, n_iter = 1000, cv = 5, verbose = True, n_jobs = -1)
best_clf_xgb_rnd = clf_xgb_rnd.fit(X_train_scaled,y_train)
clf_performance(best_clf_xgb_rnd,'XGB')"""

**AdaBoost:**

Focusing on wrongly classified samples from the previous classifier by giving it more weights in the next classifier, aiming to reduce the wrong classification results.

Assign each classifer a weight and integrate all classifiers (weighted) together to get the final answer.

**GBDT (ensembling learning):** 

Focusing on residual, the difference between previous model's result from the real answer, by establishing a new classifier on the fastest direction of the residual decreasing (gradient descent), aiming to reduce the value of residuel.

Sum up the results from all classifiers to get the final answer.

**XGBoost (ensembleing learning) :**

Similar to GBDT. Focusing on residual, the difference between previous model's result from the real answer, by establishing a new classifier on the minimizing the user-definable loss function (regularization enabled), aiming to reduce the value of residuel.

Sum up the results from all classifiers to get the final answer.

GBDT and XGBoost have similar strategies but differnt methods to minimize residual 残差




In [ ]:
xgb = XGBClassifier(random_state = 1)

param_grid = {
    'n_estimators': [450,500,550],
    'colsample_bytree': [0.75,0.8,0.85],
    'max_depth': [None],
    'reg_alpha': [1],
    'reg_lambda': [2, 5, 10],
    'subsample': [0.55, 0.6, .65],
    'learning_rate':[0.5],
    'gamma':[.5,1,2],
    'min_child_weight':[0.01],
    'sampling_method': ['uniform']
}

# colsample_bytree: the ratio of subsample of features for every tree constructed
# reg_alpha: weight for L1 regularization. 
# reg_lambda: weight for L2 regularization.
# sub_sample: the ratio of training data for every boosting iteration.
# learning_rate: step size shrinkage for gradient descrent.
# gamma: minimum loss reduction before making further partition on a leaf node.
# min_child_weight: minimum sum of instance rate for a child before it makes further partition.
# sampling_method: ways to sample data from training instances (uniform: equal probability)


clf_xgb = GridSearchCV(xgb, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_xgb = clf_xgb.fit(X_train_scaled,y_train)
clf_performance(best_clf_xgb,'XGB')


In [ ]:
y_hat_xgb = best_clf_xgb.best_estimator_.predict(X_test_scaled).astype(int)
xgb_submission = {'PassengerId': test.PassengerId, 'Survived': y_hat_xgb}
submission_xgb = pd.DataFrame(data=xgb_submission)
submission_xgb.to_csv('xgb_submission3.csv', index=False)

## Model Additional Ensemble Approaches 

Soft voting/Majority Rule classifir for unfitted estimators.

1) Experimented with a hard voting classifier of three estimators (KNN, SVM, RF) (81.6%)

2) **Experimented with a soft voting classifier of three estimators (KNN, SVM, RF) (82.3%) (Best Performance)**

3) Experimented with soft voting on all estimators performing better than 80% except xgb (KNN, RF, LR, SVC) (82.9%)

4) Experimented with soft voting on all estimators including XGB (KNN, SVM, RF, LR, XGB) (83.5%)

**Hard Voting:** determine final results using predicted class labels on majority rule basis.

**Soft Voting:** determine final results using argmax of the sums of the predicted probabilties (recommended for ensembling of calibrated ensembler). The label with the highest average argmax sum of total probabilities will be considered as the final result. 

In [ ]:
best_lr = best_clf_lr.best_estimator_
best_knn = best_clf_knn.best_estimator_
best_svc = best_clf_svc.best_estimator_
best_rf = best_clf_rf.best_estimator_
best_xgb = best_clf_xgb.best_estimator_

voting_clf_hard = VotingClassifier(estimators = [('knn',best_knn),('rf',best_rf),('svc',best_svc)], voting = 'hard') 
voting_clf_soft = VotingClassifier(estimators = [('knn',best_knn),('rf',best_rf),('svc',best_svc)], voting = 'soft') 
voting_clf_all = VotingClassifier(estimators = [('knn',best_knn),('rf',best_rf),('svc',best_svc), ('lr', best_lr)], voting = 'soft') 
voting_clf_xgb = VotingClassifier(estimators = [('knn',best_knn),('rf',best_rf),('svc',best_svc), ('xgb', best_xgb),('lr', best_lr)], voting = 'soft')

print('voting_clf_hard :',cross_val_score(voting_clf_hard,X_train,y_train,cv=5))
print('voting_clf_hard mean :',cross_val_score(voting_clf_hard,X_train,y_train,cv=5).mean())

print('voting_clf_soft :',cross_val_score(voting_clf_soft,X_train,y_train,cv=5))
print('voting_clf_soft mean :',cross_val_score(voting_clf_soft,X_train,y_train,cv=5).mean())

print('voting_clf_all :',cross_val_score(voting_clf_all,X_train,y_train,cv=5))
print('voting_clf_all mean :',cross_val_score(voting_clf_all,X_train,y_train,cv=5).mean())

print('voting_clf_xgb :',cross_val_score(voting_clf_xgb,X_train,y_train,cv=5))
print('voting_clf_xgb mean :',cross_val_score(voting_clf_xgb,X_train,y_train,cv=5).mean())


Voting Classifier can assign weights for each classifier, which will affect the soft voting argmax of the sums of the predicted probabilities. 

Voting Classifier can also be implemeted with GridSearchCV to fine-tune the parameters of the estimotors embedded in the voting classifier. 

In [ ]:
#in a soft voting classifier you can weight some models more than others. I used a grid search to explore different weightings
#no new results here
params = {'weights' : [[1,1,1],[1,2,1],[1,1,2],[2,1,1],[2,2,1],[1,2,2],[2,1,2]]}

vote_weight = GridSearchCV(voting_clf_soft, param_grid = params, cv = 5, verbose = True, n_jobs = -1)
best_clf_weight = vote_weight.fit(X_train_scaled,y_train)
clf_performance(best_clf_weight,'VC Weights')
voting_clf_sub = best_clf_weight.best_estimator_.predict(X_test_scaled)


In [ ]:
#Make Predictions 
voting_clf_hard.fit(X_train_scaled, y_train)
voting_clf_soft.fit(X_train_scaled, y_train)
voting_clf_all.fit(X_train_scaled, y_train)
voting_clf_xgb.fit(X_train_scaled, y_train)

best_rf.fit(X_train_scaled, y_train)
y_hat_vc_hard = voting_clf_hard.predict(X_test_scaled).astype(int)
y_hat_rf = best_rf.predict(X_test_scaled).astype(int)
y_hat_vc_soft =  voting_clf_soft.predict(X_test_scaled).astype(int)
y_hat_vc_all = voting_clf_all.predict(X_test_scaled).astype(int)
y_hat_vc_xgb = voting_clf_xgb.predict(X_test_scaled).astype(int)

In [ ]:
#convert output to dataframe 
final_data = {'PassengerId': test.PassengerId, 'Survived': y_hat_rf}
submission = pd.DataFrame(data=final_data)

final_data_2 = {'PassengerId': test.PassengerId, 'Survived': y_hat_vc_hard}
submission_2 = pd.DataFrame(data=final_data_2)

final_data_3 = {'PassengerId': test.PassengerId, 'Survived': y_hat_vc_soft}
submission_3 = pd.DataFrame(data=final_data_3)

final_data_4 = {'PassengerId': test.PassengerId, 'Survived': y_hat_vc_all}
submission_4 = pd.DataFrame(data=final_data_4)

final_data_5 = {'PassengerId': test.PassengerId, 'Survived': y_hat_vc_xgb}
submission_5 = pd.DataFrame(data=final_data_5)

final_data_comp = {'PassengerId': test.PassengerId, 'Survived_vc_hard': y_hat_vc_hard, 'Survived_rf': y_hat_rf, 'Survived_vc_soft' : y_hat_vc_soft, 'Survived_vc_all' : y_hat_vc_all,  'Survived_vc_xgb' : y_hat_vc_xgb}
comparison = pd.DataFrame(data=final_data_comp)

In [ ]:
#track differences between outputs 
comparison['difference_rf_vc_hard'] = comparison.apply(lambda x: 1 if x.Survived_vc_hard != x.Survived_rf else 0, axis =1)
comparison['difference_soft_hard'] = comparison.apply(lambda x: 1 if x.Survived_vc_hard != x.Survived_vc_soft else 0, axis =1)
comparison['difference_hard_all'] = comparison.apply(lambda x: 1 if x.Survived_vc_all != x.Survived_vc_hard else 0, axis =1)


In [ ]:
comparison.difference_hard_all.value_counts()

In [ ]:
#prepare submission files 
submission.to_csv('submission_rf.csv', index =False)
submission_2.to_csv('submission_vc_hard.csv',index=False)
submission_3.to_csv('submission_vc_soft.csv', index=False)
submission_4.to_csv('submission_vc_all.csv', index=False)
submission_5.to_csv('submission_vc_xgb2.csv', index=False)